In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.metrics import accuracy_score

In [3]:
url = "https://raw.githubusercontent.com/jmaxwallace/Capstone/main/model_base.csv"
df = pd.read_csv(url)

display(df.columns)

Index(['Interaction index', 'Player index', 'Opponent index', 'Player name',
       'Opponent name', 'Score', 'Score per turn', 'Initial cooperation', '0',
       '1',
       ...
       '190', '191', '192', '193', '194', '195', '196', '197', '198', '199'],
      dtype='object', length=208)

In [4]:
interaction = df['Interaction index']
turns = df.drop(columns = ['Interaction index', 'Player index', 'Opponent index', 'Player name',
       'Opponent name', 'Score', 'Score per turn', 'Initial cooperation'])

display(turns)

,0,1,2,3,4,5,6,7,8,9,...,190,191,192,193,194,195,196,197,198,199
0,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
1,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
2,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,0
3,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,0,0
4,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
235,0,1,1,0,1,0,1,0,1,0,...,1,1,0,0,1,0,1,0,1,1
236,1,0,0,1,1,0,0,0,0,1,...,1,0,1,1,0,0,1,0,1,0
237,0,0,1,0,1,1,0,0,1,0,...,0,1,1,1,0,0,0,1,1,1
238,1,1,1,0,1,1,0,0,1,0,...,0,0,0,0,0,0,0,1,1,1


In [7]:
turn_values = turns.values

games = turn_values.reshape(-1, 2, turns.shape[1])

display(games)

array([[[1, 1, 1, ..., 1, 1, 1],
        [1, 1, 1, ..., 1, 1, 1]],

       [[1, 1, 1, ..., 1, 1, 0],
        [1, 1, 1, ..., 1, 0, 0]],

       [[1, 1, 1, ..., 1, 1, 1],
        [1, 1, 1, ..., 1, 1, 1]],

       ...,

       [[1, 0, 0, ..., 1, 0, 1],
        [0, 1, 1, ..., 0, 1, 1]],

       [[1, 0, 0, ..., 0, 1, 0],
        [0, 0, 1, ..., 1, 1, 1]],

       [[1, 1, 1, ..., 1, 1, 1],
        [0, 0, 0, ..., 0, 1, 0]]], dtype=int64)

In [9]:
print(games.shape)

(120, 2, 200)


In [11]:
N = 5  # Number of previous moves to use as features

X, y = [], []

for game in games:  # Iterate over each game (2-row pairs)
    player_moves, opponent_moves = game  # Separate strategy & opponent moves

    for t in range(N, len(opponent_moves) - 1):  
        X.append(opponent_moves[t-N:t])  # Take opponent's last N moves
        y.append(opponent_moves[t+1])  # Predict opponent's next move

# Convert to NumPy arrays
X, y = np.array(X), np.array(y)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [27]:
# Initialize and train the logistic regression model
model = LogisticRegression()
model.fit(X_train, y_train)

# Predict on test set
y_pred = model.predict(X_test)

# Evaluate accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"Model Accuracy: {accuracy:.4f}")

Model Accuracy: 0.8378


In [29]:
score = df['Score'].values

score = score.reshape(-1, 2)

display(score)

array([[600, 600],
       [595, 600],
       [600, 600],
       [600, 600],
       [600, 600],
       [595, 600],
       [600, 600],
       [600, 600],
       [525, 525],
       [597, 597],
       [257, 262],
       [205, 210],
       [462, 467],
       [436, 441],
       [440, 445],
       [596, 596],
       [604, 594],
       [600, 595],
       [600, 595],
       [596, 596],
       [600, 595],
       [600, 595],
       [594, 554],
       [233, 203],
       [228, 228],
       [224, 224],
       [270, 265],
       [543, 163],
       [594, 144],
       [600, 600],
       [600, 600],
       [600, 600],
       [594, 604],
       [600, 600],
       [600, 600],
       [552, 632],
       [ 15, 980],
       [450, 700],
       [531, 646],
       [552, 632],
       [319, 784],
       [312, 792],
       [600, 600],
       [600, 600],
       [595, 600],
       [600, 600],
       [600, 600],
       [585, 565],
       [ 99, 654],
       [424, 574],
       [489, 559],
       [507, 567],
       [397,

In [45]:
N = 20  # Number of previous moves to use as features

X, y = [], []

for game_idx, game in enumerate(games):  # Iterate over each game
    for strategy_idx in range(2):  # Each game has 2 strategies
        strategy_moves = game[strategy_idx]  # Get strategy's moves
        final_score = score[game_idx, strategy_idx]  # Get corresponding final score
        
        # Use last N moves as features
        X.append(strategy_moves[-N:])  
        y.append(final_score)  


# Convert to NumPy arrays
X, y = np.array(X), np.array(y)

# Train-test split (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [49]:
from sklearn.metrics import mean_squared_error

# Initialize and train the regression model
model = LinearRegression()
model.fit(X_train, y_train)

# Predict on test set
y_pred = model.predict(X_test)

# Evaluate using Mean Squared Error (MSE)
mse = mean_squared_error(y_test, y_pred)
print(f"Model MSE: {mse:.4f}")

Model MSE: 12730.0521


In [84]:
display(y_pred)

array([326.87336801, 559.8859426 , 559.8859426 , 559.8859426 ,
       504.3304998 , 326.87336801, 485.59373617, 559.8859426 ,
       559.8859426 , 326.87336801, 564.82833649, 326.87336801,
       559.8859426 , 559.8859426 , 559.8859426 , 530.72431206,
       326.87336801, 559.8859426 , 559.8859426 , 326.87336801,
       525.78191817, 559.8859426 , 356.03499855, 554.40346958,
       326.87336801, 559.8859426 , 564.82833649, 480.65134229,
       377.48641692, 559.8859426 , 360.97739244, 326.87336801,
       326.87336801, 326.87336801, 559.8859426 , 559.8859426 ,
       559.8859426 , 504.3304998 , 559.8859426 , 326.87336801,
       372.00394389, 559.8859426 , 326.87336801, 360.97739244,
       326.87336801, 559.8859426 , 406.10796833, 554.40346958])

In [50]:
display(y_test)

array([462, 600, 600, 600, 397, 209, 402, 600, 600, 235, 554, 612, 600,
       600, 600, 459, 300, 600, 600, 296, 232, 597, 439, 596, 221, 600,
       595, 689, 450, 600,  99, 318, 467, 601, 597, 600, 554, 163, 632,
       293, 391, 600, 218, 525, 211, 600,  92, 596], dtype=int64)

In [148]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.preprocessing import StandardScaler

# Define N (last N moves to use as features)
N = 10  

X, y = [], []

for game_idx, game in enumerate(games):  
    for strategy_idx in range(2):  
        strategy_moves = game[strategy_idx]  # Get strategy's moves
        final_score = score[game_idx, strategy_idx]  # Get corresponding final score
        
        X.append(strategy_moves[-N:])  # Use last N moves
        y.append(final_score)  

# Convert to NumPy arrays
X, y = np.array(X), np.array(y)

# Normalize input data
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [146]:
# Define the model
model = keras.Sequential([
    layers.Input(shape=(N,)),  # Explicit input layer
    layers.Dense(32, activation='relu'),  # First hidden layer
    layers.Dense(16, activation='relu'),  # Second hidden layer
    layers.Dense(1)  # Output layer (predicts final score)
])

# Compile the model
model.compile(optimizer='adam', loss='mse', metrics=['mae'])

# Print model summary
model.summary()

Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_12 (Dense)                │ (None, 32)             │           352 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_13 (Dense)                │ (None, 16)             │           528 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_14 (Dense)                │ (None, 1)              │            17 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 897 (3.50 KB)

 Trainable params: 897 (3.50 KB)

 Non-trainable params: 0 (0.00 B)

In [130]:
history = model.fit(X_train, y_train, epochs=1000, batch_size=16, validation_data=(X_test, y_test))

Epoch 1/1000
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 30736.6855 - mae: 126.8811 - val_loss: 20637.3672 - val_mae: 95.3018
Epoch 2/1000
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 32620.5605 - mae: 131.4919 - val_loss: 20545.4219 - val_mae: 94.3761
Epoch 3/1000
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 26640.1387 - mae: 116.7551 - val_loss: 20454.5098 - val_mae: 94.3203
Epoch 4/1000
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 32203.7383 - mae: 131.6807 - val_loss: 20372.5703 - val_mae: 93.6214
Epoch 5/1000
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 31971.4473 - mae: 125.1261 - val_loss: 20290.0156 - val_mae: 93.2892
Epoch 6/1000
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 32918.2422 - mae: 131.0803 - val_loss: 20205.4121 - val_mae: 92.6900
Epoch 7/1000
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 31555.0254 - mae: 126.5090 - val_loss: 20104.2344 - val_mae: 92.5001
Epoch 8/1000
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 36966.3477 - mae: 139.2129 - val_loss

In [132]:
# Evaluate the model
test_loss, test_mae = model.evaluate(X_test, y_test)
print(f"Test MAE: {test_mae:.4f}")

2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - loss: 13819.0977 - mae: 81.1812
Test MAE: 84.3022


In [134]:
# Define N (last N moves to use as features)
N = 10  

X, y = [], []

for game_idx, game in enumerate(games):  
    for strategy_idx in range(2):  
        strategy_moves = game[strategy_idx]  # Get strategy's moves
        opponent_moves = game[1 - strategy_idx]  # Get opponent's moves
        final_score = score[game_idx, strategy_idx]  # Get corresponding final score

        combined_moves = np.hstack([strategy_moves[-N:], opponent_moves[-N:]])  # Combine both strategies
        X.append(combined_moves)
        y.append(final_score)  

# Convert to NumPy arrays
X, y = np.array(X), np.array(y)

# Normalize input data
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [136]:
from tensorflow import keras
from tensorflow.keras import layers

# Define the neural network model
model = keras.Sequential([
    layers.Input(shape=(2 * N,)),  # Input layer: 2N features
    layers.Dense(64, activation='relu'),  # More neurons since input size is larger
    layers.Dense(32, activation='relu'),
    layers.Dense(1)  # Output layer (predicts final score)
])

# Compile the model
model.compile(optimizer='adam', loss='mse', metrics=['mae'])

# Print model summary
model.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_6 (Dense)                 │ (None, 64)             │         1,344 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,457 (13.50 KB)

 Trainable params: 3,457 (13.50 KB)

 Non-trainable params: 0 (0.00 B)

In [138]:
history = model.fit(X_train, y_train, epochs=1000, batch_size=8, validation_data=(X_test, y_test))

Epoch 1/1000
24/24 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 206361.4531 - mae: 416.0335 - val_loss: 245247.7500 - val_mae: 466.2374
Epoch 2/1000
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 211293.6562 - mae: 425.7723 - val_loss: 241526.0781 - val_mae: 462.3748
Epoch 3/1000
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 225115.7812 - mae: 442.6911 - val_loss: 233951.5625 - val_mae: 454.4076
Epoch 4/1000
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 205219.4844 - mae: 419.1486 - val_loss: 218760.4844 - val_mae: 438.1353
Epoch 5/1000
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 203955.0312 - mae: 420.6122 - val_loss: 193617.7969 - val_mae: 409.4883
Epoch 6/1000
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 168786.2500 - mae: 370.8356 - val_loss: 158211.6094 - val_mae: 365.0125
Epoch 7/1000
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 153861.8750 - mae: 347.5515 - val_loss: 117116.0000 - val_mae: 304.9247
Epoch 8/1000
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 100553.1953 - ma

In [140]:
# Evaluate the model
test_loss, test_mae = model.evaluate(X_test, y_test)
print(f"Test MAE: {test_mae:.4f}")

2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - loss: 5681.4214 - mae: 48.3636
Test MAE: 47.5887


In [53]:
N = 20  # Number of previous moves to use as features

X, y = [], []

for game_idx, game in enumerate(games):  
    for strategy_idx in range(2):  
        strategy_moves = game[strategy_idx]  # Get strategy's moves
        opponent_moves = game[1 - strategy_idx]  # Get opponent's moves
        final_score = score[game_idx, strategy_idx]  # Get corresponding final score

        combined_moves = np.hstack([strategy_moves[-N:], opponent_moves[-N:]])  # Combine both strategies
        X.append(combined_moves)
        y.append(final_score) 

# Convert to NumPy arrays
X, y = np.array(X), np.array(y)

# Train-test split (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [55]:
from sklearn.metrics import mean_squared_error

# Initialize and train the regression model
model = LinearRegression()
model.fit(X_train, y_train)

# Predict on test set
y_pred = model.predict(X_test)

# Evaluate using Mean Squared Error (MSE)
mse = mean_squared_error(y_test, y_pred)
print(f"Model MSE: {mse:.4f}")

Model MSE: 5187.5535


In [15]:
print(games[1])

[[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
  1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
  1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
  1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
  1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
  1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0]
 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
  1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
  1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
  1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
  1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
  1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0]]


In [19]:
games.shape

(120, 2, 200)

In [31]:
print(y_test)

[462 600 600 600 397 209 402 600 600 235 554 612 600 600 600 459 300 600
 600 296 232 597 439 596 221 600 595 689 450 600  99 318 467 601 597 600
 554 163 632 293 391 600 218 525 211 600  92 596]


In [35]:
print(y_train)

[220 600 238 631  97 604 477 106 600 354 980 526 565 552 274 600 559 600
 600 600 364 539 221 600 445 236 700 312 600 594 657 272 654 604 234 576
 230 596 600 595 440 600 251 599 239 224 521 266 531 595 259 594 552 251
 436 379 263 406 600 600 595 646 233 489 272 367 234 600 600 105 267 424
 600 246 600 205 257 276 522 594 600 464 441 134 600 284 604 505 411 600
 595 502 502 586 552 136 594 594 411 600 289 210 245 212 784 596 459 203
 585 136 595 372 416 600 547 265 595 367 300 266 228 234 319 600 600 276
 595 600 792 239 474 600 600 600 600 208 596 525 595  15 323 600 209 218
 600 543 583 224 200 293 200 648 440 260 600 600 228 595 262 144 633 239
 467 546 281 257 595 201 437 600 270 129 596 600 567 574 664 600 543 450
 258 265 289 491 257 438 632 338 600 600 230 507]


In [37]:
print(combined_moves)

[1 0 0 0 0 1 1 0 0 0 0 0 1 1 1 0 1 0 1 0 0 1 1 0 0 1 1 0 1 1 0 0 0 0 0 0 0
 1 1 1]


In [39]:
print(X)

[[1 1 1 ... 1 1 1]
 [1 1 1 ... 1 1 1]
 [1 1 1 ... 1 0 0]
 ...
 [0 0 0 ... 0 1 0]
 [0 1 1 ... 0 1 0]
 [1 0 0 ... 1 1 1]]


In [43]:
print(X[1])

[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1]


In [47]:
X.shape

(240, 40)